In [2]:
import os
import numpy as np
import pandas as pd
import glob
import re
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [ ]:
class Dataset_Stock_hour(Dataset):
    def __init__(self, args, root_path, flag='train', size=None,
                 features='S', data_path='../stock_data/train_set/ST世茂.csv',
                 target='open', scale=True, timeenc=0, freq='d', seasonal_patterns=None):
        # size [seq_len, label_len, pred_len]
        self.args = args
        # info
        if size == None:
            self.seq_len = 4
            self.label_len = 1
            self.pred_len = 1
        else:
            self.seq_len = size[0]
            self.label_len = size[1]
            self.pred_len = size[2]
        # init
        assert flag in ['train', 'test', 'val']
        type_map = {'train': 0, 'val': 1, 'test': 2}
        self.set_type = type_map[flag]

        self.features = features
        self.target = target
        self.scale = scale
        self.timeenc = timeenc
        self.freq = freq

        self.root_path = root_path
        self.data_path = data_path
        self.__read_data__()

    def __read_data__(self):
        print("Loading data ...")
        self.scaler = StandardScaler()
        df_raw = pd.read_csv(os.path.join(self.root_path,
                                          self.data_path))

        border1s = [0, 16 * 20 - self.seq_len, 16 * 20 + 6 * 20 - self.seq_len]
        border2s = [16 * 20, 16 * 20 + 6 * 20, 16 * 20 + 8 * 20]
        border1 = border1s[self.set_type]
        border2 = border2s[self.set_type]

        if self.features == 'M' or self.features == 'MS':
            cols_data = df_raw.columns[1:]
            df_data = df_raw[cols_data]
        elif self.features == 'S':
            df_data = df_raw[[self.target]]

        if self.scale:
            train_data = df_data[border1s[0]:border2s[0]]
            self.scaler.fit(train_data.values)
            data = self.scaler.transform(df_data.values)
        else:
            data = df_data.values

        # ... 处理时间标签
        df_stamp = df_raw[['date']][border1:border2]
        df_stamp['date'] = pd.to_datetime(df_stamp.date)
        if self.timeenc == 0:
            df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
            df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
            df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
            df_stamp['hour'] = df_stamp.date.apply(lambda row: row.hour, 1)
            print("df_stamp:", df_stamp.head(20))
            data_stamp = df_stamp.drop(['date'], axis=1).values
        elif self.timeenc == 1:
            # data_stamp = time_features(pd.to_datetime(df_stamp['date'].values), freq=self.freq)
            data_stamp = data_stamp.transpose(1, 0) 

        self.data_x = data[border1:border2]
        self.data_y = data[border1:border2]

        # if self.set_type == 0 and self.args.augmentation_ratio > 0:
            # self.data_x, self.data_y, augmentation_tags = run_augmentation_single(self.data_x, self.data_y, self.args)

        self.data_stamp = data_stamp
        print("Data shape: ", self.data_x.shape)
        print("Stamp shape: ", self.data_stamp.shape)
        print("Sample:", self.data_x[0], self.data_y[0], self.data_stamp[0])

    def __getitem__(self, index):
        s_begin = index
        s_end = s_begin + self.seq_len
        r_begin = s_end - self.label_len
        r_end = r_begin + self.label_len + self.pred_len

        seq_x = self.data_x[s_begin:s_end]
        seq_y = self.data_y[r_begin:r_end]
        seq_x_mark = self.data_stamp[s_begin:s_end]
        seq_y_mark = self.data_stamp[r_begin:r_end]

        return seq_x, seq_y, seq_x_mark, seq_y_mark

    def __len__(self):
        return len(self.data_x) - self.seq_len - self.pred_len + 1

    def inverse_transform(self, data):
        return self.scaler.inverse_transform(data)
    

# timeenc选择0
test = Dataset_Stock_hour(None, './', flag='train')

test.__getitem__(0)


Loading data ...
df_stamp:          date  month  day  weekday  hour
0  2018-01-02      1    2        1     0
1  2018-01-03      1    3        2     0
2  2018-01-04      1    4        3     0
3  2018-01-05      1    5        4     0
4  2018-01-08      1    8        0     0
5  2018-01-09      1    9        1     0
6  2018-01-10      1   10        2     0
7  2018-01-11      1   11        3     0
8  2018-01-12      1   12        4     0
9  2018-01-15      1   15        0     0
10 2018-01-16      1   16        1     0
11 2018-01-17      1   17        2     0
12 2018-01-18      1   18        3     0
13 2018-01-19      1   19        4     0
14 2018-01-22      1   22        0     0
15 2018-01-23      1   23        1     0
16 2018-01-24      1   24        2     0
17 2018-01-25      1   25        3     0
18 2018-01-26      1   26        4     0
19 2018-01-29      1   29        0     0
Data shape:  (320, 1)
Stamp shape:  (320, 4)
Sample: [0.6170156] [0.6170156] [1 2 1 0]


C:\Users\何锦洋\AppData\Local\Temp\ipykernel_46408\1751489724.py:59: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['month'] = df_stamp.date.apply(lambda row: row.month, 1)
C:\Users\何锦洋\AppData\Local\Temp\ipykernel_46408\1751489724.py:60: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['day'] = df_stamp.date.apply(lambda row: row.day, 1)
C:\Users\何锦洋\AppData\Local\Temp\ipykernel_46408\1751489724.py:61: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  df_stamp['weekday'] = df_stamp.date.apply(lambda row: row.weekday(), 1)
C:\Users\何锦洋\AppData\Local\Temp\ipykernel_46408\1751489724.py:

(array([[0.6170156 ],
        [0.78608729],
        [0.83219776],
        [0.83219776]]),
 array([[0.83219776],
        [1.61607559]]),
 array([[1, 2, 1, 0],
        [1, 3, 2, 0],
        [1, 4, 3, 0],
        [1, 5, 4, 0]], dtype=int64),
 array([[1, 5, 4, 0],
        [1, 8, 0, 0]], dtype=int64))